# Projeto ML - Titanic espacial:
Projeto de Machine Learning cujo objetivo é identificar quais membros da hipotética tripulação foram foram transportados para outra dimensão. A base de dados pertence a uma competição do Kaggle, no link: https://www.kaggle.com/competitions/spaceship-titanic

## Parte 1 - Carregamento dos dados:

### Importando bibliotecas:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
%matplotlib inline

### Importando classes e métodos:

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

### Carregando base de dados:

In [ ]:
train_df = pd.read_csv(r'../input/spaceship-titanic/train.csv')

In [ ]:
train_df.head()

### Entendendo o problema:

A ideia do projeto é utilizar um modelo de machine learning para classificar se um indivíduo foi transportado ou não. A coluna **"Transported"** é responsável por definir se alguém foi transportado ou não, sendo essa a coluna target do projeto, os dados restantes serão trabalhados e preparados para alimentar um modelo de classificação que consiga identificar as relações entre as features e o target.

## Parte 2 - Exploração dos dados:

### Conhecendo a base de dados:

#### Proporções:

In [ ]:
print(f'O dataframe tem {train_df.shape[0]} linhas e {train_df.shape[1]} colunas')

O dataframe não é muito grande, pois tem menos de 10 mil linhas. Já as colunas devem ser analisadas com atenção, pois 14 colunas podem se multiplicar durante a preparação dos dados para modelagem.

#### Estatísticas descritivas:

In [ ]:
train_df.describe()

É possível perceber uma grande disparidade entre as distribuições de valores, havendo uma variação gigante no último quartil enquanto os outros são basicamente populados unicamente por valores nulos, sendo esse padrão recorrente em todas as colunas com excessão de **"Age"**. Portanto as medidas sensíveis a outliers são inutilizáveis visto que representarão apenas os valores do último quartil. 

In [ ]:
train_df.info()

É possível perceber o equilíbrio entre a quantidade de features categóricas e numéricas.

#### Target:

In [ ]:
target = train_df['Transported'].value_counts()
print(f'Há {target[0]} valores verdadeiros e {target[1]} valores falsos')

A proporção entre True and False na coluna target é equilibrada.

#### Identificando os valores nulos:

In [ ]:
missing_values = train_df.isna().sum()/len(train_df)
lista_missing_values = missing_values.index.to_list()

In [ ]:
for coluna in lista_missing_values:
    print(f'A coluna {coluna} tem {round(missing_values[coluna],3)}% dos valores faltando')

A maioria das colunas tem uma porcentagem de cerca de 2% de valores faltando.

#### Separando as colunas por categoria:

In [ ]:
categorical_cols = train_df.select_dtypes(include=object).columns.to_list()
print(f'As colunas categóricas são {categorical_cols}')

In [ ]:
train_df[categorical_cols].head()

In [ ]:
numeric_cols = train_df.select_dtypes(include=np.number).columns.to_list()
print(f'As colunas numéricas são {numeric_cols}')

In [ ]:
train_df[numeric_cols].head()

### Visualizando os dados:

#### Distribuição dos dados:

##### Problema encontrado:
* As colunas numéricas fora **"Age"** possuem uma dispersão de dados muito grande, tornando os histogramas irrelevantes para a visualização dos dados, portanto será necessário fazer uma transformação nos dados, no caso será utilizada uma tranformação logarítimica 

##### Transformando as colunas dispersas:

In [ ]:
visu_df = train_df.copy() # Copiando o dataframe para mudar os dados sem atrapalhar a modelagem

In [ ]:
def zero_pra_1(number): #Função para tranformar os 0 em 1
    if number == 0:
        return 1
    else:
        return number

In [ ]:
for index in range(1,6):
    visu_df[numeric_cols[index]] = visu_df[numeric_cols[index]].apply(zero_pra_1)  # Todos os zeros foram transformados em 1

##### Histogramas:

In [ ]:
plt.figure(figsize = (10,5), dpi = 150)
sns.histplot(data = train_df,
             x = 'Age',
             kde = True,
             hue = 'Transported',
             multiple = 'stack')
plt.title('Distribuição dos dados em Age', fontsize=15)
plt.ylabel('Quantidade', fontsize=12)
plt.xlabel('Age', fontsize=12)
plt.show()

É possível analisar que a idade não é um fator realmente relevante para definir se alguém foi transportado ou não, pois as distribuições de idade entre indivíduos transportados e não-transportados são muito semelhantes. Com exceção da idade de 0 anos, que apresenta uma quantidade maior de transportados do que não-transportados.

In [ ]:
lista_de_cores = ['#be2596','#96be25','#9925be','#0abcf3','#F5f377']
lista_index = list(range(1,6))

plt.figure(figsize = (10,10),dpi = 300)
plt.suptitle('Histogramas')
for index,cor in zip(lista_index, lista_de_cores):
    plt.subplot(3,2,index)
    sns.histplot(data = visu_df, 
                 x = numeric_cols[index], 
                 kde = True, 
                 log_scale = True, 
                 color = cor)
    plt.ylabel('Quantidade')
plt.tight_layout()
plt.show()

É possível confirmar as ideias concebidas anteriormente, pois é evidente uma dispersão extrema dos dados, sendo em sua maioria concentrados no número 0 e totalmente espalhados a partir de 1. Esse tipo de distribuição é um problema que precisa ser resolvido no projeto. Outro detalhe é que as distribuições são praticamente iguais.

#### Correlação entre os dados:

##### Mapa de calor:

In [ ]:
plt.figure(figsize = (10,5), dpi = 150)
sns.heatmap(data = visu_df.corr(), 
            annot = True, 
            cmap = 'magma')
plt.title('Mapa de calor das correlações entre as colunas')
plt.tight_layout()
plt.show()

Não há correlações fortes entre as colunas, porém é preciso notar que as colunas **"VRDeck"**, **"Spa"** e **"RoomService"** apresentam uma fraca correlação negativa com a coluna target **"Transported"**.

##### Gráficos de barras:

In [ ]:
sns.set(font_scale = 2)
sns.catplot(data = train_df,
            col = 'CryoSleep',
            x = 'Transported',
            palette = 'plasma',
            kind = 'count',
            aspect = 1,
            height = 10)
plt.tight_layout()
plt.show()

Há uma relação relevante entre **"CryoSleep"** e **"Transported"**, pois é evidente que dado True em **"CryoSleep"** há uma porcentagem muito maior de pessoas transportadas, enquanto em False há uma quantidade maior de pessoas não-transportadas.

In [ ]:
sns.catplot(data = train_df,
            col = 'VIP',
            x = 'Transported',
            palette = 'spring',
            kind = 'count',
            aspect = 1,
            height = 10)
plt.tight_layout()
plt.show()

A relação entre VIP e Transported é irrelevante, pois é visível que a proporção de Trues e Falses é semelhante, havendo uma quantidade próxima de transportados e não-transportados entre VIPs e não VIPs. A maioria absoluta não é VIP.

In [ ]:
sns.catplot(data = train_df,
              col = 'Destination',
              x = 'Transported',
              palette = 'icefire',
              kind = 'count',
              aspect = 1,
              height = 10)
plt.tight_layout()
plt.show()

O destino dos tripulantes apresenta uma pequena correlação a feature target, visto que pessoas indo a TRAPPIST-1e tendem a ser menos transportadas, já pessoas com destino para 55 Cancri e apresentam uma maior quantidade de transportados, enquanto pessoas viajando para PSO J318.5-22 estão divididas igualmente entre transportados e não-transportados. A maioria das pessoas está viajando para TRAPPIST-1e.

In [ ]:
sns.catplot(data = train_df,
            col = 'HomePlanet',
            x = 'Transported',
            palette = 'rocket',
            kind = 'count',
            aspect = 1,
            height = 10)
plt.tight_layout()
plt.show()
sns.reset_orig()

As pessoas na Terra são percentualmente menos transportadas, em Marte a diferença entre transportados e não-transportados é irrelevante e as pessoas de Europa tem mais propensão a serem transportadas. A grande maioria das pessoas viajando é da Terra.

#### Identificando outliers:

In [ ]:
plt.figure(figsize = (10,10), dpi = 300)
plt.suptitle('Boxplots')
for index, col in zip([1,2,3,4,5], numeric_cols):
    plt.subplot(3,2,index)
    sns.boxplot(data = train_df, 
                y = col)
plt.tight_layout()
plt.show()

É possível concluir que os outliers são de fato um problema nesse dataset, pois são presentes em grandes quantidades em diversas colunas.

## Parte 3 - Tratamento dos dados:

### Feature Engineering:

####  Completando os valores faltando:

##### Colunas numéricas:

In [ ]:
train_df[numeric_cols].head()

In [ ]:
train_df[numeric_cols].isna().sum() # Dados faltando

Para completar as colunas, dividerei as colunas numéricas, imputando os dados faltando de **"Age"** a parte, pois as distribuições dos outros dados são semelhantes e totalmente diferentes de **"Age"**

###### Age:

In [ ]:
def impute_age(df): # Função que imputa os números na coluna "Age"
    impute = df['Age'].dropna().sample(df['Age'].isnull().sum()).to_list()
    indexes = df[df['Age'].isnull()].index.to_list()
    df.loc[indexes,'Age'] = impute
    return df

In [ ]:
impute_age(train_df).head(3) # Completando o dataset

In [ ]:
train_df['Age'].isnull().sum() # Dados faltando

In [ ]:
plt.figure(figsize = (10,5), dpi = 150)
sns.histplot(train_df['Age'], 
             kde=True)
plt.title('Nova distribuição de Age')
plt.ylabel('Densidade')

Como é possível perceber, após a imputação de dados, a distribuição dos dados em **"Age"** continua igual, graças ao método de Random Sampling aplicado.

###### Medidas de Luxo:

In [ ]:
imputer = SimpleImputer(strategy = 'most_frequent') # Preparando o imputer
train_df[numeric_cols[1:]] = imputer.fit_transform(train_df[numeric_cols[1:]]) # Completando as colunas

In [ ]:
train_df[numeric_cols[1:]].isnull().sum() # Dados faltando

Todas os dados faltando foram completados por 0, que é o número mais frequente nas 5 colunas, consistindo de basicamente 3/4 dos dados, por isso sua adição não distorceu a distribuição, que já era concentrada no 0.

##### Colunas categóricas:

In [ ]:
train_df[categorical_cols].head()

* É possível presumir o valor faltando da coluna **"CryoSleep"**, pois quando os passegeiros estão em sono criogênco eles não consomem.
* **"Name"** e **"Cabin"** não vão entrar no modelo, no máximo serão extraídas features a partir dessas colunas.
* As outras colunas serão imputados com o dado de maior frequência.

In [ ]:
train_df[categorical_cols].isna().sum() # Dados faltando

In [ ]:
colunas = ['VIP','Destination','HomePlanet'] # Completando essas colunas
train_df[colunas] = imputer.fit_transform(train_df[colunas])

In [ ]:
train_df[categorical_cols].isna().sum() # Dados faltando

CryoSleep vai ser imputada posteriormente, pois será criada uma feature nova que captura todos os gastos com luxos, facilitando o processo.

#### Criando novas features:

##### Intuições: 
1. Durante a exploração de dados foi identificada uma grande disparidade entre os dados quanto ao consumo de luxos, ocorrendo uma clara separação em 2 grupos, os que consomem e os que não consomem. Havendo cerca de 3/4 que tem um gasto em luxos de 0, e um grupo de 1/4 que gasta com luxos, entre esses 1/4 há uma grande variação nos gastos.

2. É possível extrair os grupos de passegeiros a partir da coluna **"PassengerID"**.

3. A coluna **"Cabin"** entrega 2 informações interessantes, o deck no qual o passageiro está hospedado e o lado no qual a cabine está alocada.

##### Desenvolvendo a primeira intuição:

In [ ]:
total_gasto = 0 # Somando todos os gastos em luxo
for index in range(1,6):
    total_gasto += train_df[numeric_cols[index]]

In [ ]:
train_df['SomaLuxos'] = total_gasto # Colocando a agregação na base de dados

In [ ]:
train_df['SomaLuxos'] = np.where(train_df['SomaLuxos'] == 0,1,train_df['SomaLuxos']) 
# Transformando 0 em 1, para poder usar o log

###### Análise:

In [ ]:
plt.figure(figsize = (5,4), dpi = 150)
sns.histplot(data = train_df,
             x = 'SomaLuxos',
             log_scale = True,
             hue = 'Transported',
             multiple = 'stack',
             kde = True,
             palette='pastel')
plt.ylabel('Quantidade')
plt.title('Distribuição de SomaLuxos')
plt.show()

Nas distribuições separadas era possível identificar uma discrepância grande nos gastos, porém agora com seus valores agregados, é possível eliminar o fator das predileções particulares de cada tripulante por determinados serviços, conseguindo uma separação perfeita entre pessoas que gastam em luxos e pessoas que não gastam. A maior descoberta dessa agregação é o fato de que pessoas com gastos em luxos são menos transportadas do que pessoas que não gastam, sendo a diferença muito grande para ser uma mera coincidência.

In [ ]:
def separar_grupos(number): # Essa é a função para formar a nova feature
    if number <= 5:
        return 0 
    else:
        return 1

In [ ]:
train_df['ConsomeLuxo'] = train_df['SomaLuxos'].apply(separar_grupos) # Criando a feature

Como havia uma clara separação entre quem consome e quem não consome luxos, acredito que seja mais inteligente formar uma feature categórica que determina o consumo de luxo, para identificar a relação entre o consumo de luxo e ser transportado.

In [ ]:
sns.set(font_scale = 2)
sns.catplot(data = train_df,
            col = 'ConsomeLuxo',
            x = 'Transported',
            palette = 'bwr',
            kind = 'count',
            aspect = 1,
            height = 10)
plt.tight_layout()
plt.show()
sns.reset_orig()

É visível a relação entre as 2 variavéis, havendo uma quantidade muito grande de transportados entre quem não consome luxos, e dentre quem consome, uma minoria foi transportada.

###### CryoSleep:
Agora que existe uma feature que define o consumo de luxos, é possível aumentar as probabilidades de completar os dados com exatidão.

In [ ]:
def definir_cryo(n): # Função que define se uma pessoa está em sono criogênico ou não
    if n >= 1:
        return False 
    else:
        return True

In [ ]:
def impute_cryo(df): # Imputando os dados faltando em "CryoSleep"
    serie = df.loc[df['CryoSleep'].isna() == True,'ConsomeLuxo'].apply(definir_cryo)
    impute = serie.to_list()
    indexes = df[df['CryoSleep'].isnull()].index.to_list()
    df.loc[indexes,'CryoSleep'] = impute
    return df 

In [ ]:
impute_cryo(train_df).head(2) # imputando os dados

In [ ]:
train_df['CryoSleep'].isnull().sum()

Agora todas as features originais relevantes foram imputadas.

##### Desenvolvendo a segunda intuição:

In [ ]:
train_df['Grupos'] = train_df['PassengerId'].str[0:4] # Cortando os grupos dos ids

In [ ]:
n_groups = pd.DataFrame([train_df['Grupos'].value_counts(ascending=False)]) #Criando a feature 
n_df = n_groups.T
n_df.reset_index(inplace = True)
n_df.rename(columns = {'Grupos':'N_grupo','index':'Grupos'}, inplace = True)

In [ ]:
train_df = train_df.merge(n_df, how = 'left', on = "Grupos") # juntando a nova feature no dataframe

Através dos dados dos ids dos passageiros foi possível deduzir o tamanho do grupo ao qual fazem parte, gerando a feature **"N_grupo"** que representa a quantidade de pessoas no grupo fora o indivíduo.

###### Análise:

In [ ]:
ordem = train_df[['N_grupo',
                  'Transported']].groupby('N_grupo').mean().sort_values('Transported',
                                                                          ascending = False).index.to_list()
order = [str(n) for n in ordem] # Ordem decrescente 

In [ ]:
plt.figure(figsize = (10,5), dpi = 300)
sns.barplot(data = train_df,
            x = 'Transported',
            y = train_df['N_grupo'].astype(str),
            palette = 'gnuplot2_r',
            order = order)
plt.title('% de transportados por N° no grupo', fontsize = 15)
plt.ylabel('N° no grupo', fontsize = 15)
plt.xlabel('% Transportados', fontsize = 15)
plt.tight_layout()
plt.show()

É possível presumir que, dado que o dataset é equilibrado na porcentagem de transportados, o número de pessoas em cada grupo não parece ser um fator levado em conta no momento do transporte. Pois, todos os valores apresentam uma média próxima de 50 % de transportados, ainda mais levando em conta que a maioria dos intervalos de confiança se encontra.

##### Desenvolvendo a terceira intuição:

###### Deck:

In [ ]:
train_df['Deck'] = train_df.Cabin.str[0] # Capturando a feature 
train_df['Deck']

In [ ]:
deck_df = train_df[['Deck','Transported']].groupby('Deck').mean().sort_values('Transported', ascending = False)

In [ ]:
plt.figure(figsize = (10,5), dpi = 150)
sns.barplot(data = deck_df,
            y = deck_df.index, 
            x = 'Transported',
            palette = 'deep')
plt.title('% transportados por Deck', fontsize = 15)
plt.ylabel('Deck', fontsize = 15)
plt.xlabel('% transportado', fontsize = 15)
plt.tight_layout()
plt.show()

A nova feature **"Deck"**, acrescenta informações interessantes ao dataframe, pois há uma grande variação na porcentagem de transportados entre os decks, como no deck "T" em que houve uma porcentagem muito pequena de transportados, ao contrário do deck "B" em que a maioria foi transportado.

In [ ]:
train_df['Deck'] = imputer.fit_transform(train_df[['Deck']])

In [ ]:
train_df['Deck'].isna().sum()

###### Side:

In [ ]:
train_df['Side'] = train_df.Cabin.str[-1] # Capturando a feature
train_df['Side']

In [ ]:
side_df = train_df[['Side','Transported']].groupby('Side').mean().sort_values('Transported', ascending = False)

In [ ]:
plt.figure(figsize = (10,5), dpi = 150)
sns.barplot(data = side_df,
            y = side_df.index, 
            x = 'Transported',
            palette = 'gist_heat')
plt.title('% transportados por Side', fontsize = 15)
plt.ylabel('Side', fontsize = 15)
plt.xlabel('% transportado', fontsize = 15)
plt.tight_layout()
plt.show()

A feature **"Side"** aparentemente denota uma porcentagem maior de transportados do lado "S", porém não chega a ser uma diferença grande o bastante para ser confirmada como algo além de aleatoriedade.

In [ ]:
def impute_side(df): # Função que imputa os números na coluna "Side"
    impute = df['Side'].dropna().sample(df['Side'].isnull().sum()).to_list()
    indexes = df[df['Side'].isnull()].index.to_list()
    df.loc[indexes,'Side'] = impute
    return df

In [ ]:
impute_side(train_df) # Imputando dados em Side
train_df['Side'].isna().sum()

#### Nova separação das features:

In [ ]:
categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination','VIP',
                    'Deck', 'Side', 'N_grupo', 'ConsomeLuxo']

In [ ]:
numeric_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'SomaLuxos']

Nova separação das features, excluindo as features **"Grupos"**, pois já é representada pela feature **"N_grupos"**, **"Name"**,**"PassengerId"** e **"Cabin"** foram excluídas também pois não agregam informações ao modelo.

#### Escalando os dados:

In [ ]:
train_df[numeric_cols].head()

In [ ]:
scaler = RobustScaler() # Escalonando os dados
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])

In [ ]:
train_df[numeric_cols].head()

Agora as features contínuas estão devidamente escaladas. O RobustScaler foi utilizado pois há uma quantidade muito grande de outliers na base de dados.

#### Codificação dos dados categóricos:

In [ ]:
train_df[categorical_cols].head()

##### Colunas com muitas variáveis:

In [ ]:
train_df[['N_grupo','Deck']].head()

In [ ]:
deck_df = train_df[['Deck','Transported']].groupby('Deck').mean().sort_values('Transported', ascending = True)
display(deck_df) # DF com o deck e a % correspondente de ser transportado

In [ ]:
labels_deck = deck_df.index.to_list()
deck_mapper = {d:n for d,n in zip(labels_deck,range(0,len(labels_deck)))} # Mapeador do deck
print(deck_mapper)

In [ ]:
n_df = train_df[['N_grupo','Transported']].groupby('N_grupo').mean().sort_values('Transported', ascending = False)
display(n_df) # DF com o n° no grupo e a % correspondente de ser transportado

In [ ]:
labels_n = n_df.index.to_list()
n_mapper = {d:n for d,n in zip(labels_n,range(0,len(labels_n)))} # Mapeador do N_grupo
print(n_mapper)

In [ ]:
train_df['Deck'] = train_df['Deck'].map(deck_mapper) # Transformando os dados
train_df['N_grupo'] = train_df['N_grupo'].map(n_mapper)

In [ ]:
train_df[['Deck','N_grupo']].head()

##### Colunas binárias:

In [ ]:
train_df[['CryoSleep','VIP','ConsomeLuxo','Side']].head()

In [ ]:
bool_mapper = {False:0, True:1} # Mapeadores para transformar os dados
side_mapper = {'P':0, 'S':1}

In [ ]:
train_df['CryoSleep'] = train_df['CryoSleep'].map(bool_mapper) # Transformando as features
train_df['VIP'] = train_df['VIP'].map(bool_mapper)
train_df['Side'] = train_df['Side'].map(side_mapper)

In [ ]:
train_df[['CryoSleep','VIP','ConsomeLuxo','Side']].head()

##### One hot encoding:

In [ ]:
train_df[['HomePlanet','Destination']].head()

In [ ]:
encoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore') # Treinando o modelo 
encoder.fit(train_df[['HomePlanet','Destination']])

In [ ]:
features_names = list(encoder.get_feature_names_out()) # Nomes das novas features
train_df[features_names] = encoder.transform(train_df[['HomePlanet','Destination']]) # Colocando no modelo

Agora todas as features estão prontas para serem selecionadas para o modelo.

### Feature Selection:

In [ ]:
y = train_df['Transported'] # Retirando o target do dataframe
train_df = train_df.drop('Transported',axis = 1)

In [ ]:
# Retirando as colunas que não servem para o modelo
features_df = train_df.drop(['PassengerId','HomePlanet','Cabin','Destination','Grupos','Name'], axis = 1)

In [ ]:
n_cols = len(features_df.columns)
print(f'O número de features no modelo é de {n_cols}')

In [ ]:
features_df.isna().sum()

Há 19 features no modelo, dentre elas selecionaremos as que entrarão no modelo.

#### Seleção por ganho de informação:

In [ ]:
inf_array = mutual_info_classif(features_df,y)
gain_df = pd.DataFrame(inf_array, index = features_df.columns, columns = ['Gain'])
gain_df.sort_values('Gain', ascending = False)

É notável que as features com mais ganho de informação são as features relacionadas ao consumo de luxos e **"CryoSleep"**, seguido por **"Deck"** e **"HomePlanet"**, a partir disso as features apresentam pouco ou nenhum ganho informacional.

#### Seleção por chi2:

In [ ]:
chi2_df = features_df[['CryoSleep','VIP','ConsomeLuxo','N_grupo',
             'Deck','Side','HomePlanet_Earth','HomePlanet_Europa',
             'HomePlanet_Mars','Destination_55 Cancri e',
             'Destination_PSO J318.5-22','Destination_TRAPPIST-1e']]

In [ ]:
f_selector = SelectKBest(score_func = chi2, k = 10)
f_selector.fit(chi2_df,y)
f_array = f_selector.scores_
f_df = pd.DataFrame(f_array, index = chi2_df.columns, columns = ['Score'] )
f_df.sort_values('Score',ascending = False)

Na seleção por Chi2 as features com mais importância são as mesmas de antes, porém com uma diferença na ordem.

#### Escolhendo as features:

In [ ]:
features_df.drop(["Spa", "RoomService", "VRDeck", 
                  "ShoppingMall","FoodCourt"], axis = 1, inplace = True)

Essas features estão sendo retiradas do dataset, pois elas compõem através de uma soma a feature **"SomaLuxos"**. As features restantes serão utilizadas na modelagem.

In [ ]:
features_df.head()

In [ ]:
features_model = len(features_df.columns)
print(f'Serão usadas {features_model} features nos modelos.')

#### Salvando o trabalho:

In [ ]:
features_df.to_csv('features_df.csv', index = False)
y.to_csv('y.csv', index = False)

## Parte 4 - Selecionando o modelo:

### Carregando o dataset:

In [ ]:
train_df = pd.read_csv('features_df.csv')
y = pd.read_csv('y.csv')
display(train_df.head(), y.head())

In [ ]:
y = np.where(y == False, 0 , 1) # Convertendo o target

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_df,y, random_state = 20) # Separando os dados em treino e validação

### Selecionando os modelos:

#### Modelo escolhido:

Será testado o modelo de  **XGBoost Classifier**.

#### Montando o modelo:

#### XGBClassifier:

In [ ]:
xgb = XGBClassifier(random_state = 20, n_jobs = -1, verbosity = 0) # Modelo

In [ ]:
xgb.fit(X_train, y_train.ravel()) # Treinando o modelo
xgb_predict = xgb.predict(X_val) # Previsão

In [ ]:
print('Classification report')
print(classification_report(xgb_predict, y_val))

O modelo em sua forma padrão, apresenta uma precisão de 74%, se tratando da precisão, ela se destaca na label 0 (Não-transportado) com 81%, já na label 1 (Transportado) apresenta 70% de precisão, um resultado mediano. Já quanto ao recall, na label 0 é de 70%, e na label 1 é de 78%, um resultado interessante. 

###### Hyperparameter Tuning:

In [ ]:
# Hiperparâmetros para serem testados:
param_grid_xgb = {'n_estimators' : [10,30,60,100,150,200,300,400,500,600,1000,2000],
                  'max_depth' : [5,10,20,40,80,120,200,300,400,500,600,700,1000,2000],
                  'learning_rate' : list(np.linspace(0,1,10)),
                  'gamma' : [1,10,100,1000],
                  'reg_lambda' : list(np.linspace(0,100,100)),
                  'max_leaves' : [5,10,20,40,80,120,200,300,400,600,1000,2000,3000]}

In [ ]:
# Modelo de otimização:
xgb_random = RandomizedSearchCV(xgb, 
                                param_grid_xgb, 
                                n_jobs = -1, 
                                random_state = 20, 
                                n_iter = 30, 
                                verbose = False)

In [ ]:
xgb_random.fit(X_train, y_train.ravel()) 
xgb_random_predict = xgb_random.predict(X_val)

In [ ]:
print('Classification report')
print(classification_report(xgb_random_predict, y_val))

Após utilizar uma técnica de otimização de hiperparâmetros o modelo performou de forma semelhante, porém houve um agravante, as melhores métricas do modelo anterior estão melhores, enquanto as que estavam ruins pioraram. Porém, o uso dessa técnica tinha como objetivo se aproximar da melhor escolha de hiperparâmetros, visto que agora pode-se montar outra grade, uma mais próxima do ideal, e assim a otimização vai ser melhor concluída.

In [ ]:
# Nova grande de parâmetros:
gridsearch_params = {'reg_lambda': [7,8,9],
                    'n_estimators': [250,300,350],
                    'max_leaves': [10,20,30],
                    'max_depth': [30,40,50],
                    'learning_rate': [0.5,0.6,0.7],
                    'gamma': [5,10,15]}

In [ ]:
# Novo modelo de otimização:
xgb_otimized = GridSearchCV(xgb, 
                            gridsearch_params,
                            n_jobs = -1, 
                            cv = 2,
                            verbose = True)

In [ ]:
xgb_otimized.fit(X_train, y_train.ravel()) 
xgb_otimized_predict = xgb_otimized.predict(X_val)

In [ ]:
print('Classification report')
print(classification_report(xgb_otimized_predict, y_val))

Esse é o modelo final, apresentando uma precisão maior que a dos outros modelos, com uma média de 75%, houve uma melhora de 1% nas métricas de recall, e um aumento total de 3% na precisão.

###### Salvando o modelo:

In [ ]:
joblib.dump({xgb_otimized},'model.joblib') # Salvando o modelo

## Parte 5 - Pipeline de dados:

### Carregando os dados de teste:

In [ ]:
test_df = pd.read_csv(r'../input/spaceship-titanic/test.csv')
test_df.head()
sample_df = pd.read_csv(r'../input/spaceship-titanic/sample_submission.csv')

In [ ]:
model = joblib.load('model.joblib') # Carregando o modelo
model = list(model)[0]

### Funções de impute:

In [ ]:
def get_n_cols(df): # Função para separar as colunas
    numeric_cols = df.select_dtypes(include = np.number).columns.to_list()
    return numeric_cols

In [ ]:
def impute_age(df): # Completa os dados em "Age" 
    impute = df['Age'].dropna().sample(df['Age'].isnull().sum()).to_list()
    indexes = df[df['Age'].isnull()].index.to_list()
    df.loc[indexes,'Age'] = impute
    return df

In [ ]:
def simple_imputer(df, n_cols): # Simple Imputer    
    imputer = SimpleImputer(strategy = 'most_frequent')
    df[n_cols[1:]] = imputer.fit_transform(df[n_cols[1:]])
    colunas = ['VIP','Destination','HomePlanet']
    df[colunas] = imputer.fit_transform(df[colunas])
    return df

In [ ]:
def impute_side(df): # Completa os dados em "Side"
    impute = df['Side'].dropna().sample(df['Side'].isnull().sum()).to_list()
    indexes = df[df['Side'].isnull()].index.to_list()
    df.loc[indexes,'Side'] = impute
    return df

In [ ]:
def impute_cryo(df): # Completa os dados em "CryoSleep"
    def definir_cryo(n):
        if n >= 1:
            return False 
        else:
            return True
    serie = df.loc[df['CryoSleep'].isna() == True,'SomaLuxos'].apply(definir_cryo)
    impute = serie.to_list()
    indexes = df[df['CryoSleep'].isnull()].index.to_list()
    df.loc[indexes,'CryoSleep'] = impute
    return df 

### Funções para criar as features:

In [ ]:
def soma_luxos(df, n_cols): # Criando a feature "SomaLuxos"
    total_gasto = 0 
    for index in range(1,6):
        total_gasto += df[n_cols[index]]
    df['SomaLuxos'] = total_gasto 
    return df

In [ ]:
def consumo_luxo(df): # Criando a feature "Consumoluxo"
    def separar_grupos_consumo(n): 
        if n <= 5:
            return 0 
        else:
            return 1
    df['ConsomeLuxo'] = df['SomaLuxos'].apply(separar_grupos_consumo)
    return df

In [ ]:
def n_grupos(df): # Criando a feature "N_grupos"
    df['Grupos'] = df['PassengerId'].str[0:4]
    n_groups = pd.DataFrame([df['Grupos'].value_counts(ascending=False)]) 
    n_df = n_groups.T
    n_df.reset_index(inplace = True)
    n_df.rename(columns = {'Grupos':'N_grupo','index':'Grupos'}, inplace = True)
    df = df.merge(n_df, how = 'left', on = "Grupos")
    df.drop('Grupos', axis = 1, inplace = True)
    return df

In [ ]:
def deck_side(df): # Criando as features "Deck" e "Side"
    imputer = SimpleImputer(strategy = 'most_frequent')
    df['Deck'] = df.Cabin.str[0] 
    df['Deck'] = imputer.fit_transform(df[['Deck']])
    df['Side'] = df.Cabin.str[-1]
    return df

### Funções de encoding:

In [ ]:
def label_encoder(df): # Label encoder
    deck_mapper = {'T': 0, 'E': 1, 'D': 2, 'F': 3, 'A': 4, 'G': 5, 'C': 6, 'B': 7}
    n_mapper = {4: 0, 6: 1, 3: 2, 5: 3, 7: 4, 2: 5, 1: 6, 8: 7}

    df['Deck'] = df['Deck'].map(deck_mapper) 
    df['N_grupo'] = df['N_grupo'].map(n_mapper)
    return df

In [ ]:
def binary_encoder(df): # Encoder para features binárias
    bool_mapper = {False:0, True:1} 
    side_mapper = {'P':0, 'S':1}

    df['CryoSleep'] = df['CryoSleep'].map(bool_mapper) # Transformando as features
    df['VIP'] = df['VIP'].map(bool_mapper)
    df['Side'] = df['Side'].map(side_mapper)
    return df

In [ ]:
def one_hot_encoding(df): # One Hot Encoder
    encoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore') 
    encoder.fit(df[['HomePlanet','Destination']])
    features_names = list(encoder.get_feature_names_out())
    df[features_names] = encoder.transform(df[['HomePlanet','Destination']])
    return df

In [ ]:
def scaler(df): # Scaler
    scaler = RobustScaler()
    numeric_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'SomaLuxos']
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    return df

### Unificando as funções:

In [ ]:
def transform_df(df): # União de todas as funções
    n_cols = get_n_cols(df)
    df = impute_age(df)
    df = simple_imputer(df, n_cols)
    df = soma_luxos(df,n_cols)
    df = consumo_luxo(df)
    df = impute_cryo(df)
    df = n_grupos(df)
    df = deck_side(df)
    df = impute_side(df)
    df = label_encoder(df)
    df = binary_encoder(df)
    df = one_hot_encoding(df)
    df = scaler(df)
    df.drop(["Spa", "RoomService", "VRDeck", "ShoppingMall","FoodCourt",
             'HomePlanet','Cabin','Destination','PassengerId', 'Name'], axis = 1, inplace = True)
    return df

### Teste final no modelo:

In [ ]:
test_df = transform_df(test_df) # Aplicando todas as funções no dataframe
model_predict = model.predict(test_df)

In [ ]:
predict = pd.Series(model_predict, name ='Transported').apply(lambda x: bool(x)) # Transformando os valores em booleanos
df = pd.concat([sample_df['PassengerId'],predict], axis = 1)

In [ ]:
df.to_csv('final_predict.csv', index = False) # Salvando o modelo

O modelo teve o desempenho final de 70,75% de precisão.